In [1]:
from transformers.models.bert.modeling_bert import BertEncoder, BertPooler, BertEmbeddings, BaseModelOutputWithPoolingAndCrossAttentions
from transformers import BertConfig, BertModel
import torch
from torch import nn

2023-01-25 14:03:13.901553: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 14:03:14.028419: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64
2023-01-25 14:03:14.028445: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-25 14:03:14.694518: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot ope

In [2]:
import numpy as np 
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences    

import pandas as pd

# from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, f1_score
import pickle
import torch
from torchcrf import CRF

from sklearn.model_selection import train_test_split

import transformers
from transformers import BertForTokenClassification, AdamW, BertModel, BertConfig
from transformers import BertTokenizer, BertConfig

from transformers import AutoTokenizer
from transformers import get_linear_schedule_with_warmup, get_constant_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

from tqdm import tqdm, trange
import numpy as np 

import torch.nn as nn

import numpy as np
# from seqeval.metrics import classification_report as classification_report_seqeval
# from seqeval.metrics import classification_report

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device)

MAX_LEN = 128
BATCH_SIZE = 32

import torch.nn.functional as F
log_soft = F.log_softmax


In [7]:
class BertEmbeddingsV2(BertEmbeddings):
    def __init__(self, config):
        super().__init__(config)
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)
        max_number_of_pos_tags = 34
        self.pos_tag_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        
        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        # position_ids (1, len position emb) is contiguous in memory and exported when serialized
        self.register_buffer("position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)))
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")

    def forward(self, input_ids=None, pos_tag_ids=None, token_type_ids=None, position_ids=None, inputs_embeds=None, past_key_values_length=0):
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]
        # print("=======>", input_shape)

        if position_ids is None:
            position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]

        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=self.position_ids.device)

        if inputs_embeds is None:
            inputs_embeds = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        pos_tag_embeddings = self.pos_tag_embeddings(pos_tag_ids)
        
        # print("INPUT_ID_SHAPE => " ,inputs_embeds.shape)
        # print("token_type_tag_ids1_SHAPE => " ,token_type_embeddings.shape)
        # print("pos_tag_ids1_SHAPE => " ,pos_tag_embeddings.shape)

        embeddings = inputs_embeds + token_type_embeddings + pos_tag_embeddings
        if self.position_embedding_type == "absolute":
            position_embeddings = self.position_embeddings(position_ids)
            embeddings += position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [8]:
class BertModelV2(BertModel):
    """
    The model can behave as an encoder (with only self-attention) as well as a decoder, in which case a layer of
    cross-attention is added between the self-attention layers, following the architecture described in `Attention is
    all you need <https://arxiv.org/abs/1706.03762>`__ by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit,
    Llion Jones, Aidan N. Gomez, Lukasz Kaiser and Illia Polosukhin.
    To behave as an decoder the model needs to be initialized with the :obj:`is_decoder` argument of the configuration
    set to :obj:`True`. To be used in a Seq2Seq model, the model needs to initialized with both :obj:`is_decoder`
    argument and :obj:`add_cross_attention` set to :obj:`True`; an :obj:`encoder_hidden_states` is then expected as an
    input to the forward pass.
    """

    def __init__(self, config, add_pooling_layer=True):
        super().__init__(config)
        self.config = config

        self.embeddings = BertEmbeddingsV2(config)
        self.encoder = BertEncoder(config)

        self.pooler = BertPooler(config) if add_pooling_layer else None

        self.init_weights()

    
    def forward(
        self,input_ids=None,attention_mask=None,token_type_ids=None,pos_tag_ids=None,position_ids=None,head_mask=None,inputs_embeds=None,encoder_hidden_states=None,encoder_attention_mask=None,
        past_key_values=None,use_cache=None,output_attentions=None,output_hidden_states=True,return_dict=None,
    ):
        r"""
        encoder_hidden_states  (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length, hidden_size)`, `optional`):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in ``[0, 1]``:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (:obj:`tuple(tuple(torch.FloatTensor))` of length :obj:`config.n_layers` with each tuple having 4 tensors of shape :obj:`(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.
            If :obj:`past_key_values` are used, the user can optionally input only the last :obj:`decoder_input_ids`
            (those that don't have their past key value states given to this model) of shape :obj:`(batch_size, 1)`
            instead of all :obj:`decoder_input_ids` of shape :obj:`(batch_size, sequence_length)`.
        use_cache (:obj:`bool`, `optional`):
            If set to :obj:`True`, :obj:`past_key_values` key value states are returned and can be used to speed up
            decoding (see :obj:`past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
            batch_size, seq_length = input_shape
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
            batch_size, seq_length = input_shape
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)
        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            pos_tag_ids=pos_tag_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        extend_out= BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )
        return encoder_outputs


In [9]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
print(1)

1


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
def read_dataset(file):
    data = pd.read_csv(file)
    return data

def split_data(data):
    X_label, POS, Y_label=[],[],[]

    for i in range(len(data)):
        words= list(data.sentence[i].split())
        pos_tags = literal_eval(data.noun_features[i])
        ner_tag = literal_eval(data.ner_features[i])
        X_label.append(words)
        POS.append(pos_tags)
        Y_label.append(ner_tag)
    return X_label, POS, Y_label
    
def create_dict(POS, y):
    pos_set = set()
    for li in POS:
        for val in li:
            pos_set.add(val)
    ner_set = set()
    for li in y:
        for val in li:
            ner_set.add(val)
    ner_set=sorted(list(ner_set))
    pos_set=sorted(list(pos_set))
    train_tag_values = sorted(list(ner_set))
    train_tag_values.append("PAD")
    pos2id={'NA': 0,'PAD': 1}
    ner2id = {t: i for i, t in enumerate(train_tag_values)}

    ind=2
    for x in pos_set:
        pos2id[x] = ind
        ind+=1
    return pos2id,ner2id,train_tag_values

def _add_subword(tokenizer, words, pos_tags, ner_tag):
    tokens = []
    pos_tag_tokens = []
    ner_tag_tokens =[]
    for word, tag, ner in zip(words, pos_tags,ner_tag ):
    # tokenize the word
        word_tokens = tokenizer.tokenize(word)
        tokens.extend(word_tokens)
    # copy the POS tag and NER tag for all word tokens2
        pos_tag_tokens.extend([tag for _ in range(len(word_tokens))])
        ner_tag_tokens.extend([ner for _ in range(len(word_tokens))])
    # tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
    # pos_tag_tokens = ['NA'] + pos_tag_tokens + ['NA']
    # ner_tag_tokens = ['NA'] + ner_tag_tokens + ['NA']
    return tokens,pos_tag_tokens,ner_tag_tokens
    

def add_subword2data(tokenizer,X,pos, Y):
    '''
            input:
                sentence = [['Ayush', 'Kumar', 'Mishra',..],....]
                text_labels = [['B-PER', 'I-PER','I-PER',..],...]

            output: 
                [['Ayush', 'Kumar', 'Mish', '##ra',..],....],
                [['B-PER', 'I-PER','I-PER','I-PER',..],...]
    '''
    tokenized_texts_and_labels = [_add_subword(tokenizer,sent,pos, labs) for sent,pos,labs in zip(X,pos, Y)]
    text = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
    pos = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
    ner = [token_label_pair[2] for token_label_pair in tokenized_texts_and_labels]
    return text,pos,ner

def padding_data(tokenizer,X_subword,pos_subword, ner_subword,pos2idx,ner2idx,MAXLEN):
    X_padding = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in X_subword],maxlen=MAXLEN, dtype="long", value=0.0,truncating="post", padding="post")
    pos_padding = pad_sequences([[pos2idx.get(l) for l in lab] for lab in pos_subword],maxlen=MAXLEN, value=pos2idx["PAD"], padding="post",dtype="long", truncating="post")
    ner_padding = pad_sequences([[ner2idx.get(l) for l in lab] for lab in ner_subword],maxlen=MAXLEN, value=ner2idx["PAD"], padding="post",dtype="long", truncating="post")
    attention_masks = [[int(i != 0) for i in ii] for ii in X_padding]
    return X_padding, pos_padding, ner_padding, attention_masks

def covert2tensor(input_ids,pos_tag_ids,ner_tag_ids,attention_mask,  mode ,device):
    if mode == 'training':
        input_ids = torch.tensor(input_ids).to(device)
        pos_tag_ids = torch.tensor(pos_tag_ids).to(device) 
        ner_tag_ids = torch.tensor(ner_tag_ids).to(device) 
        attention_mask = torch.tensor(attention_mask).to(device)
        

    elif mode =='evaluation':
        input_ids = torch.tensor(input_ids).type(torch.LongTensor).to(device)
        pos_tag_ids = torch.tensor(pos_tag_ids).type(torch.LongTensor).to(device) 
        ner_tag_ids = torch.tensor(ner_tag_ids).type(torch.LongTensor).to(device) 
        attention_mask = torch.tensor(attention_mask).type(torch.LongTensor).to(device) 
        
    return  input_ids,pos_tag_ids,ner_tag_ids,attention_mask

def Dataloader(input_ids,pos_tag_ids,ner_tag_ids,attention_mask,BATCH_SIZE, mode ,type, device):
    input_ids,pos_tag_ids,ner_tag_ids,attention_mask = covert2tensor(input_ids,pos_tag_ids,ner_tag_ids,attention_mask,  mode ,device)
    if type == 'train':
        train_data = TensorDataset(input_ids,pos_tag_ids,attention_mask,ner_tag_ids)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)
        return train_dataloader
    elif type == 'dev' or type == 'test':
        valid_data = TensorDataset(input_ids,pos_tag_ids,attention_mask,ner_tag_ids)
        valid_sampler = SequentialSampler(valid_data)
        valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE)
        return valid_dataloader

In [13]:
def create_loader(dir,tokenizer,BATCH_SIZE,MAXLEN,mode,type_,device):
    data = read_dataset(dir)
    X,POS,y = split_data(data)
    pos2id,ner2id,train_tag_values = create_dict(POS,y)
    text,pos,ner= add_subword2data(tokenizer,X,POS,y)
    X_padding, pos_subword, ner_subword, attention_masks = padding_data(tokenizer, text,pos,ner,pos2id,ner2id,MAXLEN)
    loader = Dataloader(X_padding, pos_subword, ner_subword, attention_masks,BATCH_SIZE, mode ,type_ ,device)
    return X,loader, pos2id, ner2id, train_tag_values 

In [15]:
type_ = 'train'
BATCH_SIZE=32
MAX_LEN = 128
mode = 'training'
dir_train=''
train_set, train_loader, pos2id, ner2id, train_tag_values  = create_loader(dir_train, tokenizer, BATCH_SIZE, MAX_LEN, mode, type_, device)

In [16]:
type_ = 'dev'
BATCH_SIZE=32
MAX_LEN = 128
mode = 'evaluation'
dir_val=''
val_set, dev_loader, pos2id, ner2id, dev_labels  = create_loader(dir_val, tokenizer, BATCH_SIZE, MAX_LEN, mode, type_, device)

In [ ]:
type_ = 'test'
BATCH_SIZE=32
MAX_LEN = 128
mode = 'evaluation'
test_val=''
test_set, test_loader, pos2id, ner2id, dev_labels  = create_loader(test_val, tokenizer, BATCH_SIZE, MAX_LEN, mode, type_, device)

In [ ]:
config = BertConfig.from_pretrained('nlpaueb/legal-bert-base-uncased', output_hidden_states=True)
bert_model = BertModelV2.from_pretrained('nlpaueb/legal-bert-base-uncased',config=config,add_pooling_layer=False)
print(1)

In [18]:
class BERT_CRF(nn.Module):
    def __init__(self, bert_model, num_labels):
        super(BERT_CRF, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.25)
        # 4 last of layer
        self.classifier = nn.Linear(4*768, num_labels)
        self.crf = CRF(num_labels, batch_first = True)
    
    def forward_custom(self, input_ids,b_input_mask,pos_tag_ids,b_labels):
        # outputs = self.bert(b_input_ids, attention_mask=b_input_mask)
        outputs = self.bert.forward(input_ids=input_ids,attention_mask=b_input_mask,pos_tag_ids=pos_tag_ids)
        sequence_output = torch.cat((outputs[1][-1], outputs[1][-2], outputs[1][-3], outputs[1][-4]),-1)
        sequence_output = self.dropout(sequence_output)
        
        emission = self.classifier(sequence_output) # [32,256,17]
        
        
        if b_labels is not None:
            loss = -self.crf(log_soft(emission, 2), b_labels, mask=b_input_mask.type(torch.uint8), reduction='mean')
            prediction = self.crf.decode(emission, mask=b_input_mask.type(torch.uint8))
            return [loss, prediction]
                
        else:
            prediction = self.crf.decode(emission, mask=b_input_mask.type(torch.uint8))
            return prediction

In [19]:
# class BERT_BILSTM_CRF(nn.Module):
#     def __init__(self, bert_model, num_labels):
#         super(BERT_BILSTM_CRF, self).__init__()
#         self.bert = bert_model
#         self.dropout = nn.Dropout(0.25)
#         # 4 last of layer
#         rnn_dim = 2*768
#         out_dim = 4*768
#         need_birnn = True
#         if need_birnn:
#             self.birnn = nn.LSTM(out_dim, rnn_dim, num_layers=1, bidirectional=True, batch_first=True)
#             out_dim = rnn_dim*2
#         self.classifier = nn.Linear(out_dim, num_labels)
#         self.crf = CRF(num_labels, batch_first = True)
    
#     def forward_custom(self, input_ids,b_input_mask,pos_tag_ids,b_labels):
#         outputs = self.bert.forward(input_ids=input_ids,attention_mask=b_input_mask,pos_tag_ids=pos_tag_ids)
#         sequence_output = torch.cat((outputs[1][-1], outputs[1][-2], outputs[1][-3], outputs[1][-4]),-1)
#         # sequence_output = outputs[0]
#         need_birnn = True
#         if need_birnn:
#             sequence_output, _ = self.birnn(sequence_output)
#         sequence_output = self.dropout(sequence_output)
#         emission = self.classifier(sequence_output) # [32,256,17]
        
#         if b_labels is not None:
#             loss = -self.crf(log_soft(emission, 2), b_labels, mask=b_input_mask.type(torch.uint8), reduction='mean')
#             prediction = self.crf.decode(emission, mask=b_input_mask.type(torch.uint8))
#             return [loss, prediction]
                
#         else:
#             prediction = self.crf.decode(emission, mask=b_input_mask.type(torch.uint8))
#             return prediction

In [20]:
model = BERT_CRF(bert_model, num_labels=29)
# model = BERT_CRF(bert_model, num_labels=29,dropout_rate=0.25,n_class=0,lstm_hidden_size=256)
model.to(device)
print(1)

1


In [21]:
cnt = -1
num_layer = 197 #?????
for param in model.named_parameters():
    cnt += 1
    if cnt>=num_layer:
        param[1].requires_grad = True
    else:
        param[1].requires_grad = True
    # print(cnt,param[0],'\t',param[1].requires_grad)
    

FINETUNING = True
if FINETUNING:
    param_optimizer1 = list(model.named_parameters())[:num_layer]
    param_optimizer2 = list(model.named_parameters())[num_layer:num_layer+2]
    param_optimizer3 = list(model.named_parameters())[num_layer+2:]
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer1 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 1e-5},
        {'params': [p for n, p in param_optimizer1 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0},
        
        {'params': [p for n, p in param_optimizer2 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 1e-3,
         'lr': 1e-3},
        {'params': [p for n, p in param_optimizer2 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0,
         'lr':1e-3},
        
        {'params': [p for n, p in param_optimizer3 if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 1e-3,
         'lr':4e-3},
        {'params': [p for n, p in param_optimizer3 if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0,
         'lr':4e-3}
    ]
    
optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)


In [22]:
epochs = 50
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(total_steps/10),
    num_training_steps=total_steps
)

In [23]:
model_save_path = ''
PATH=''

In [ ]:
%%time
## Store the average loss after each epoch so we can plot them.
train_loss_values, valid_loss_values = [], []
f1_max = 0
f1_list = []
hist = {}


for epoch in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_loader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)

        # input_ids1,pos_tag_ids1,ner_tag_ids1,attention_mask1 = batch
        input_ids1,pos_tag_ids1,attention_mask1,ner_tag_ids1 = batch

        # print("INPUT_ID_SHAPE => " ,input_ids1.shape)
        # print("pos_tag_ids1_SHAPE => " ,pos_tag_ids1.shape)
        # print("attention_mask1_SHAPE => " ,attention_mask1.shape)
        # print("ner_tag_ids1_SHAPE => " ,ner_tag_ids1.shape)

        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        
        
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        # input_ids,b_input_mask,pos_tag_ids,b_labels=None
        outputs = model.forward_custom(input_ids=input_ids1,b_input_mask=attention_mask1,pos_tag_ids=pos_tag_ids1,b_labels=ner_tag_ids1)
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)
    print("\nAverage train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    train_loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    model.eval()
    predictions_f1 , true_labels_f1 = [], []
    eval_loss = 0
    for batch in dev_loader:

        batch = tuple(t.to(device) for t in batch)
        # b_input_ids, b_input_mask, b_labels = batch
        # b_input_ids,pos_tag_ids1,b_labels,b_input_mask = batch
        b_input_ids,pos_tag_ids1,b_input_mask,b_labels = batch

        # print("INPUT_ID_SHAPE => " ,b_input_ids.shape)
        # print("pos_tag_ids1_SHAPE => " ,pos_tag_ids1.shape)
        # print("attention_mask1_SHAPE => " ,b_input_mask.shape)
        # print("ner_tag_ids1_SHAPE => " ,b_labels.shape)

        with torch.no_grad():
            outputs = model.forward_custom(input_ids=b_input_ids,b_input_mask=b_input_mask,pos_tag_ids=pos_tag_ids1,b_labels=b_labels)

        eval_loss += outputs[0].mean().item()
        predict_labels = outputs[1]
        label_ids = b_labels.to('cpu').numpy().tolist()
        predictions = []
        for predict_label in predict_labels:
            predictions.append(predict_label)

        for b_input_id, preds, labels in zip(b_input_ids, predictions, label_ids):
            tokens = tokenizer.convert_ids_to_tokens(b_input_id.to('cpu').numpy())

            new_tokens, new_labels, new_preds = [], [], []
            for token, label_idx, pred in zip(tokens, labels, preds):
                if token.startswith("##"):
                    new_tokens[-1] = new_tokens[-1] + token[2:]
                else:
                    new_labels.append(label_idx)
                    new_preds.append(pred)
                    new_tokens.append(token)
            for token, pred, label in zip(new_tokens, new_preds, new_labels):
                predictions_f1.extend([pred])
                true_labels_f1.extend([label])
    eval_loss = eval_loss / len(dev_loader)
    print("Validation loss:", eval_loss)
    true_labels_f1_convert = [train_tag_values[i] for i in true_labels_f1]
    predictions_f1_convert = [train_tag_values[i] for i in predictions_f1]
    f1 = f1_score(true_labels_f1_convert, predictions_f1_convert,labels= dev_labels ,average='macro')
    print(classification_report(true_labels_f1_convert, predictions_f1_convert, labels= dev_labels, digits=4))
    

    # eval_loss, f1 = evaluate(BIO_dataloader_valid,device)
    valid_loss_values.append(eval_loss)
    f1_list.append(f1)
    hist['train_loss_values'] = train_loss_values
    hist['valid_loss_values'] = valid_loss_values
    hist['f1_list'] = f1_list
    if f1 > f1_max:
      
        print(f'f1_score improved from: {f1_max:.4f} to {f1:.4f}')
        print(f'Best model saved to {PATH}')
        f1_max = f1
        # torch.save({
        #     'epoch': epoch,
        #     'f1_score': f1_max,
        #     'history': hist,
        #     'model_state_dict': model.state_dict(),
        #     'optomizer_state_dict': optimizer.state_dict()
        # }, PATH)
        torch.save(model.state_dict(), model_save_path)
                    # also save the optimizers' state
        torch.save(optimizer.state_dict(), model_save_path + '.optim')
        patience = 15
        epochs_no_improve = 0
        best_epoch = epoch
    else:
        print(f'f1_score dont improve from: {f1_max:.4f} to {f1:.4f}')
        epochs_no_improve += 1
        if epochs_no_improve < patience:
            print(f'EarlyStopping count: {epochs_no_improve}/{patience}')
        else:
            print(f'\nEarly Stopping! Total epochs: {epochs}. Best epoch: {best_epoch} with f1_score: {f1_max:.4f}')
            break

In [ ]:
predictions_f1 , true_labels_f1 = [], []
word_len_list=[]
word_list=[]

In [ ]:
type_ = 'test'
BATCH_SIZE=32
MAX_LEN = 128
mode = 'evaluation'
test_val=''
test_set, test_loader, pos2id, ner2id, dev_labels  = create_loader(test_val, tokenizer, BATCH_SIZE, MAX_LEN, mode, type_, device)

In [ ]:
predictions , true_labels = [], []
sentencs=[]

# Predict 
for batch in test_loader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  # b_input_ids, b_input_mask, b_labels = batch

  b_input_ids,pos_tag_ids1,b_input_mask,b_labels = batch

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
    # model.forward_custom(b_input_ids, b_input_mask, b_labels, token_type_ids=None)
      # Forward pass, calculate logit predictions
      outputs = model.forward_custom(input_ids=b_input_ids,b_input_mask=b_input_mask,pos_tag_ids=pos_tag_ids1,b_labels=b_labels)
  test_loss=0
  test_loss += outputs[0].mean().item()
  predict_labels = outputs[1]
  label_ids = b_labels.to('cpu').numpy().tolist()
  predictions = []
  for predict_label in predict_labels:
      predictions.append(predict_label)

  for b_input_id, preds, labels in zip(b_input_ids, predictions, label_ids):
      tokens = tokenizer.convert_ids_to_tokens(b_input_id.to('cpu').numpy())

      new_tokens, new_labels, new_preds = [], [], []
      for token, label_idx, pred in zip(tokens, labels, preds):
          if token.startswith("##"):
              new_tokens[-1] = new_tokens[-1] + token[2:]
          else:
              new_labels.append(label_idx)
              new_preds.append(pred)
              new_tokens.append(token)
      count=0
      for token, pred, label in zip(new_tokens, new_preds, new_labels):
          count+=1
          predictions_f1.extend([pred])
          true_labels_f1.extend([label])
          sentencs.extend([token])
          word_list.append(new_tokens)
      word_len_list.append(count)

In [ ]:
len(sentencs), len(predictions_f1), len(word_len_list)

In [ ]:
predictions_f1_convert = [train_tag_values[i] for i in predictions_f1]


In [ ]:
word_sentence = [i for i in sentencs]


In [ ]:
len(word_sentence), len(predictions_f1_convert)

In [ ]:
word_sentence[0],predictions_f1_convert[0]

In [ ]:
true_labels_f1_convert = [train_tag_values[i] for i in true_labels_f1]
predictions_f1_convert = [train_tag_values[i] for i in predictions_f1]


In [ ]:
count=0
for  i in range(len(true_labels_f1_convert)):
    if true_labels_f1_convert[i] !=predictions_f1_convert[i]:
        count+=1
        

In [ ]:
(len(true_labels_f1_convert)-count)/len(true_labels_f1_convert)